In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


DATA_PATH = "D:\Projects\Edge-Optimized ECG Arrhythmia Detection using TensorFlow Lite and Quantization\data\ecg.csv"
MODEL_DIR = "models/ecg_saved_model"

def load_data():
    print("Loading ECG dataset...")
    df = pd.read_csv(DATA_PATH)

    X = df.iloc[:, :-1].values.astype(np.float32)
    y = df.iloc[:, -1].values.astype(np.int32)

    return train_test_split(X, y, test_size=0.2, random_state=42)

def build_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_shape,)),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

def main():
    X_train, X_test, y_train, y_test = load_data()

    print("Building model...")
    model = build_model(input_shape=X_train.shape[1], num_classes=5)

    print("Training model...")
    model.fit(X_train, y_train, epochs=5, batch_size=32)

    print("Evaluating model...")
    loss, acc = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {acc:.4f}")

    print("Saving model in SavedModel format...")
    os.makedirs("models", exist_ok=True)
    model.export(MODEL_DIR)

    print("SavedModel exported successfully!")

if __name__ == "__main__":
    main()


Loading ECG dataset...
Building model...
Training model...
Epoch 1/5
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 951us/step - accuracy: 0.8813 - loss: 0.4417
Epoch 2/5
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9271 - loss: 0.2600
Epoch 3/5
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 954us/step - accuracy: 0.9392 - loss: 0.2158
Epoch 4/5
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 923us/step - accuracy: 0.9467 - loss: 0.1914
Epoch 5/5
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 908us/step - accuracy: 0.9503 - loss: 0.1766
Evaluating model...
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9479 - loss: 0.1896  
Test Accuracy: 0.9479
Saving model in SavedModel format...
INFO:tensorflow:Assets written to: models/ecg_saved_model\assets


INFO:tensorflow:Assets written to: models/ecg_saved_model\assets


Saved artifact at 'models/ecg_saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 187), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  2145171075536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2145171077648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2145171075344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2145171075728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2145171078224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2145171075920: TensorSpec(shape=(), dtype=tf.resource, name=None)
SavedModel exported successfully!


In [2]:
import os
import tensorflow as tf

MODEL_DIR = "models/ecg_saved_model"
TFLITE_PATH = "models/ecg_model.tflite"
TFLITE_QUANT_PATH = "models/ecg_model_quant.tflite"

def convert_model():
    print("Converting SavedModel → TFLite...")

    converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

    # Normal conversion
    tflite_model = converter.convert()
    with open(TFLITE_PATH, "wb") as f:
        f.write(tflite_model)

    print("Normal TFLite model saved:", TFLITE_PATH)

    # Quantized conversion
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    quant_model = converter.convert()

    with open(TFLITE_QUANT_PATH, "wb") as f:
        f.write(quant_model)

    print("Quantized TFLite model saved:", TFLITE_QUANT_PATH)

def main():
    os.makedirs("models", exist_ok=True)
    convert_model()

if __name__ == "__main__":
    main()


Converting SavedModel → TFLite...
Normal TFLite model saved: models/ecg_model.tflite
Quantized TFLite model saved: models/ecg_model_quant.tflite


In [5]:
import numpy as np
import tensorflow as tf
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


DATA_PATH = "D:\Projects\Edge-Optimized ECG Arrhythmia Detection using TensorFlow Lite and Quantization\data\ecg.csv"
MODEL_PATH = "models/ecg_model_quant.tflite"

def main():
    print("Loading quantized TFLite model...")
    interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    print("Loading sample ECG input...")
    df = pd.read_csv(DATA_PATH)

    X = df.iloc[:, :-1].values.astype(np.float32)
    y = df.iloc[:, -1].values.astype(np.int32)

    sample = X[0]
    sample_label = y[0]

    sample = np.expand_dims(sample, axis=0)

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], sample)

    # Run inference
    interpreter.invoke()

    # Get output tensor
    output = interpreter.get_tensor(output_details[0]['index'])
    predicted_class = np.argmax(output)

    print("Actual Label:", sample_label)
    print("Predicted Label:", predicted_class)

if __name__ == "__main__":
    main()


Loading quantized TFLite model...
Loading sample ECG input...
Actual Label: 0
Predicted Label: 0


In [4]:
import os
import time
import tensorflow as tf

NORMAL_MODEL = "models/ecg_model.tflite"
QUANT_MODEL = "models/ecg_model_quant.tflite"

def benchmark(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    start = time.time()
    for _ in range(100):
        interpreter.invoke()
    end = time.time()

    avg_latency = (end - start) / 100
    size = os.path.getsize(model_path)

    return avg_latency, size

def main():
    print("Benchmarking models...\n")

    normal_latency, normal_size = benchmark(NORMAL_MODEL)
    quant_latency, quant_size = benchmark(QUANT_MODEL)

    print("Normal Model:")
    print(" Size:", normal_size, "bytes")
    print(" Avg Latency:", normal_latency, "seconds\n")

    print("Quantized Model:")
    print(" Size:", quant_size, "bytes")
    print(" Avg Latency:", quant_latency, "seconds\n")

if __name__ == "__main__":
    main()


Benchmarking models...

Normal Model:
 Size: 59208 bytes
 Avg Latency: 0.0 seconds

Quantized Model:
 Size: 18360 bytes
 Avg Latency: 0.0 seconds

